In [226]:
%run Comment_Questions.ipynb # Exectute temporary training Parameters

# NEU Unit (Reconfiguration Map)

## Initialize Modules

###### Imports

In [227]:
# Deep Learning & ML
import tensorflow as tf
import keras as Kr
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils

# General
import numpy as np
import time

# Message Boxes
import tkinter
from tkinter import messagebox

### Define Reconfiguration Unit

1. Shifts $x \in \mathbb{R}^d$ to $x- c$; c trainable.
2. Rescales componentwise with $a * x$,  $a \in \mathbb{R}^d$ trainable.
3. Applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.  
4. Applies transformation $x \mapsto x +b$, $b \in \mathbb{R}^d$ trainable.
5. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}.$
6. Applies map $X \mapsto XA$, $A$ is a trainable $d\times d$ matrix.
7. Applies matrix exponential.
8. Multiplies output with result of (1).
9. Re-centers output to $x +c$ where $c$ is as in (1).

In [345]:
import tensorflow as tf
print('TensorFlow:', tf.__version__)

class Reconfiguration_unit(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(Reconfiguration_unit, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.weight = self.add_weight(shape=[input_shape[1],input_shape[1]], # CURRENTLY NOT A MATRIX FIX THIS!!!
                                    initializer='zeros',
                                    trainable=True)
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.location = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.scale = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        
        
        
    def call(self, x):
        # 1. Shift and scale data
        x_shift = x - self.location
        
        # 2. Rescale componentwise
        x_mod = tf.math.multiply(x_shift,self.scale)
        
        # 3. Apply bumpy function Component-wise
        x_in_abs = tf.math.abs(x_mod)
        Logic_x_leq1 = tf.math.sign(tf.keras.activations.relu(1-x_in_abs)) # Takes value 1 iff |x|<=1 else 0: since probability of |x|=1 is 0 we should be ok
        x_thresheld = Logic_x_leq1*tf.math.exp(-1/(1-tf.math.pow(x_in_abs,-1))) # Computes bump function at thresholds with previous logic
        
        # 4+5. Apply Shift (In Tangent Space) and diagonalize
        x_out = tf.expand_dims((x_in_abs + self.bias), 1) 
        
        # 6. Multiply by weight matrix (in Tangent Space)
        x_out = (x_out * self.weight) 
        
        # 7. Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # 8. Muliply by output of (1)
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # 9. Recenter Transformed Data
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out

TensorFlow: 2.1.0


Reads number of trainable parameters

In [363]:
input_layer = tf.keras.Input(shape=[2])
x  = Reconfiguration_unit()(input_layer)
for(i in range(10)):
    x = Reconfiguration_unit()(x)

SyntaxError: invalid syntax (<ipython-input-363-20eb50c5b3cd>, line 3)

### Build Reconfiguration

In [340]:
model = Sequential()
model.add(layers.Dense(256, input_shape=(784,)))
model.add(Reconfiguration_unit())

TypeError: The added layer must be an instance of class Layer. Found: <__main__.Reconfiguration_unit object at 0x7f6f0bda3310>

In [342]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras import layers
from keras.datasets import mnist
from keras import backend as K


class Antirectifier(layers.Layer):
    '''This is the combination of a sample-wise
    L2 normalization with the concatenation of the
    positive part of the input with the negative part
    of the input. The result is a tensor of samples that are
    twice as large as the input samples.

    It can be used in place of a ReLU.

    # Input shape
        2D tensor of shape (samples, n)

    # Output shape
        2D tensor of shape (samples, 2*n)

    # Theoretical justification
        When applying ReLU, assuming that the distribution
        of the previous output is approximately centered around 0.,
        you are discarding half of your input. This is inefficient.

        Antirectifier allows to return all-positive outputs like ReLU,
        without discarding any data.

        Tests on MNIST show that Antirectifier allows to train networks
        with twice less parameters yet with comparable
        classification accuracy as an equivalent ReLU-based network.
    '''

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 2
        return tuple(shape)

    def call(self, inputs):
        inputs -= K.mean(inputs, axis=1, keepdims=True)
        inputs = K.l2_normalize(inputs, axis=1)
        pos = K.relu(inputs)
        neg = K.relu(-inputs)
        return K.concatenate([pos, neg], axis=1)

# global parameters
batch_size = 128
num_classes = 10
epochs = 40

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# build the model
model = Sequential()
model.add(layers.Dense(256, input_shape=(784,)))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


60000 train samples
10000 test samples
